In [269]:
import dns.resolver
import dns.query
import dns.zone
import dns.tsigkeyring
import dns.update
import sys
import dns.ipv4
import os.path
import dns.reversename
import dns.message
import dns.rdataclass
import dns.rdatatype
import random

In [50]:
qname = dns.name.from_text('amazon.com')
q = dns.message.make_query(qname, dns.rdatatype.NS)
print('The query is:')
print(q)
print('')

The query is:
id 63047
opcode QUERY
rcode NOERROR
flags RD
;QUESTION
amazon.com. IN NS
;ANSWER
;AUTHORITY
;ADDITIONAL



In [55]:
r = dns.query.udp(q, '8.8.8.8')
print('The response is:')
print(r)
print('')

The response is:
id 63047
opcode QUERY
rcode NOERROR
flags QR RD RA
;QUESTION
amazon.com. IN NS
;ANSWER
amazon.com. 3465 IN NS pdns1.ultradns.net.
amazon.com. 3465 IN NS ns4.p31.dynect.net.
amazon.com. 3465 IN NS ns3.p31.dynect.net.
amazon.com. 3465 IN NS ns2.p31.dynect.net.
amazon.com. 3465 IN NS ns1.p31.dynect.net.
amazon.com. 3465 IN NS pdns6.ultradns.co.uk.
;AUTHORITY
;ADDITIONAL



In [58]:
print('The nameservers are:')
ns_rrset = r.find_rrset(r.answer, qname, dns.rdataclass.IN, dns.rdatatype.NS)
for rr in ns_rrset:
    print(rr.target)
print('')

The nameservers are:
pdns1.ultradns.net.
ns4.p31.dynect.net.
ns3.p31.dynect.net.
ns2.p31.dynect.net.
ns1.p31.dynect.net.
pdns6.ultradns.co.uk.



In [63]:
resolver = dns.resolver.Resolver(configure=False)
resolver.nameservers = ['8.8.8.8']
answers = dns.resolver.query('amazon.com', 'NS')
print('The nameservers are:')
for rr in answers:
    print(rr.target)

The nameservers are:
ns4.p31.dynect.net.
ns3.p31.dynect.net.
pdns1.ultradns.net.
pdns6.ultradns.co.uk.
ns2.p31.dynect.net.
ns1.p31.dynect.net.


In [2]:
def print_response(response):
    '''Print the response's answer, authority, and additional
    
    Args:
        response (dns.message.Message): a response of a single iterative DNS query
    '''
    print('\n', 'QUESTION:')
    for i in response.question:
        print(i.to_text())
    
    print('\n', 'ANSWER:')
    for i in response.answer:
        print(i.to_text())
    
    print('\n', 'AUTHORITY:')
    for i in response.authority:
        print(i.to_text())
        
    print('\n', 'ADDITIONAL:')
    for i in response.additional:
        print(i.to_text())

In [344]:
a_query = dns.message.make_query('www.google.co.jp', 'A')

In [345]:
response1 = dns.query.udp(a_query, '198.41.0.4')
print_response(response1)


 QUESTION:
www.google.co.jp. IN A

 ANSWER:

 AUTHORITY:
jp. 172800 IN NS a.dns.jp.
jp. 172800 IN NS d.dns.jp.
jp. 172800 IN NS e.dns.jp.
jp. 172800 IN NS f.dns.jp.
jp. 172800 IN NS h.dns.jp.
jp. 172800 IN NS g.dns.jp.
jp. 172800 IN NS c.dns.jp.
jp. 172800 IN NS b.dns.jp.

 ADDITIONAL:
a.dns.jp. 172800 IN A 203.119.1.1
a.dns.jp. 172800 IN AAAA 2001:dc4::1
d.dns.jp. 172800 IN A 210.138.175.244
d.dns.jp. 172800 IN AAAA 2001:240::53
e.dns.jp. 172800 IN A 192.50.43.53
e.dns.jp. 172800 IN AAAA 2001:200:c000::35
f.dns.jp. 172800 IN A 150.100.6.8
f.dns.jp. 172800 IN AAAA 2001:2f8:0:100::153
h.dns.jp. 172800 IN A 65.22.40.25
h.dns.jp. 172800 IN AAAA 2a01:8840:1ba::25
g.dns.jp. 172800 IN A 203.119.40.1
c.dns.jp. 172800 IN A 156.154.100.5
c.dns.jp. 172800 IN AAAA 2001:502:ad09::5
b.dns.jp. 172800 IN A 202.12.30.131
b.dns.jp. 172800 IN AAAA 2001:dc2::1


In [346]:
response2 = dns.query.udp(a_query, '203.119.1.1')
print_response(response2)


 QUESTION:
www.google.co.jp. IN A

 ANSWER:

 AUTHORITY:
google.co.jp. 86400 IN NS ns3.google.net.
google.co.jp. 86400 IN NS ns4.google.net.
google.co.jp. 86400 IN NS ns2.google.net.
google.co.jp. 86400 IN NS ns1.google.net.

 ADDITIONAL:


In [93]:
response3 = dns.query.udp(a_query, '220.181.36.234')
print_response(response3)


 QUESTION:
www.163.com. IN A

 ANSWER:
www.163.com. 600 IN CNAME www.163.com.lxdns.com.

 AUTHORITY:

 ADDITIONAL:


In [66]:
def get_cname_from_rrset(rrset):
    '''Get CNAME from a RRset (from ANSWER section)
    
    Args:
        rrset (A DNS RRset): contains an CNAME to be extracted
        
    Returns:
        CNAME (str): the CNAME in the RRset
    '''
    try:
        item = rrset.items[0]
        return item.to_text()
    except Exception as e:
        print('Oops! Some issue with rrset: ', e)

In [107]:
b_query = dns.message.make_query('www.163.com.lxdns.com', 'A')

In [108]:
response4 = dns.query.udp(b_query, '199.9.14.201')
print_response(response4)


 QUESTION:
www.163.com.lxdns.com. IN A

 ANSWER:

 AUTHORITY:
com. 172800 IN NS d.gtld-servers.net.
com. 172800 IN NS a.gtld-servers.net.
com. 172800 IN NS b.gtld-servers.net.
com. 172800 IN NS f.gtld-servers.net.
com. 172800 IN NS e.gtld-servers.net.
com. 172800 IN NS l.gtld-servers.net.
com. 172800 IN NS i.gtld-servers.net.
com. 172800 IN NS m.gtld-servers.net.
com. 172800 IN NS h.gtld-servers.net.
com. 172800 IN NS g.gtld-servers.net.
com. 172800 IN NS j.gtld-servers.net.
com. 172800 IN NS c.gtld-servers.net.
com. 172800 IN NS k.gtld-servers.net.

 ADDITIONAL:
a.gtld-servers.net. 172800 IN A 192.5.6.30
b.gtld-servers.net. 172800 IN A 192.33.14.30
c.gtld-servers.net. 172800 IN A 192.26.92.30
d.gtld-servers.net. 172800 IN A 192.31.80.30
e.gtld-servers.net. 172800 IN A 192.12.94.30
f.gtld-servers.net. 172800 IN A 192.35.51.30
g.gtld-servers.net. 172800 IN A 192.42.93.30
h.gtld-servers.net. 172800 IN A 192.54.112.30
i.gtld-servers.net. 172800 IN A 192.43.172.30
j.gtld-servers.net. 1728

In [161]:
len(response5.additional)

0

In [250]:
response5 = dns.query.udp(b_query, '192.55.83.30')
print_response(response5)


 QUESTION:
www.163.com.lxdns.com. IN A

 ANSWER:

 AUTHORITY:
lxdns.com. 172800 IN NS dns2.lxdns.info.
lxdns.com. 172800 IN NS dns4.lxdns.info.
lxdns.com. 172800 IN NS dns1.lxdns.org.
lxdns.com. 172800 IN NS dns3.lxdns.org.
lxdns.com. 172800 IN NS dns5.lxdns.org.

 ADDITIONAL:


In [162]:
def get_ns_from_authority(response):
    rrsets = response.authority[0]
    return rrsets.items

In [163]:
ns = get_ns_from_authority(response5)

In [164]:
ns

[<DNS IN NS rdata: dns2.lxdns.info.>,
 <DNS IN NS rdata: dns4.lxdns.info.>,
 <DNS IN NS rdata: dns1.lxdns.org.>,
 <DNS IN NS rdata: dns3.lxdns.org.>,
 <DNS IN NS rdata: dns5.lxdns.org.>]

In [153]:
rrset = response5.authority[0]
items = rrset.items

In [158]:
for item in items:
    print(item.to_text())

dns2.lxdns.info.
dns4.lxdns.info.
dns1.lxdns.org.
dns3.lxdns.org.
dns5.lxdns.org.


In [159]:
c_query = dns.message.make_query('dns2.lxdns.info.', 'A')
response6 = dns.query.udp(c_query, '198.41.0.4')
print_response(response6)


 QUESTION:
dns2.lxdns.info. IN A

 ANSWER:

 AUTHORITY:
info. 172800 IN NS a0.info.afilias-nst.info.
info. 172800 IN NS a2.info.afilias-nst.info.
info. 172800 IN NS b0.info.afilias-nst.org.
info. 172800 IN NS b2.info.afilias-nst.org.
info. 172800 IN NS c0.info.afilias-nst.info.
info. 172800 IN NS d0.info.afilias-nst.org.

 ADDITIONAL:
a0.info.afilias-nst.info. 172800 IN A 199.254.31.1
a2.info.afilias-nst.info. 172800 IN A 199.249.113.1
b0.info.afilias-nst.org. 172800 IN A 199.254.48.1
b2.info.afilias-nst.org. 172800 IN A 199.249.121.1
c0.info.afilias-nst.info. 172800 IN A 199.254.49.1
d0.info.afilias-nst.org. 172800 IN A 199.254.50.1
a0.info.afilias-nst.info. 172800 IN AAAA 2001:500:19::1
a2.info.afilias-nst.info. 172800 IN AAAA 2001:500:41::1
b0.info.afilias-nst.org. 172800 IN AAAA 2001:500:1a::1
b2.info.afilias-nst.org. 172800 IN AAAA 2001:500:49::1
c0.info.afilias-nst.info. 172800 IN AAAA 2001:500:1b::1
d0.info.afilias-nst.org. 172800 IN AAAA 2001:500:1c::1


In [117]:
response7 = dns.query.udp(c_query, '199.254.31.1')
print_response(response7)


 QUESTION:
dns2.lxdns.info. IN A

 ANSWER:

 AUTHORITY:
lxdns.info. 86400 IN NS ns1.lxdns.info.
lxdns.info. 86400 IN NS ns3.lxdns.info.
lxdns.info. 86400 IN NS ns2.lxdns.info.
lxdns.info. 86400 IN NS ns4.lxdns.info.
lxdns.info. 86400 IN NS ns5.lxdns.info.

 ADDITIONAL:
ns1.lxdns.info. 86400 IN A 183.131.192.82
ns2.lxdns.info. 86400 IN A 125.46.22.158
ns3.lxdns.info. 86400 IN A 113.107.57.74
ns4.lxdns.info. 86400 IN A 122.138.54.37
ns5.lxdns.info. 86400 IN A 14.119.111.147


In [251]:
response8 = dns.query.udp(c_query, '183.131.192.82')
print_response(response8)


 QUESTION:
dns2.lxdns.info. IN A

 ANSWER:
dns2.lxdns.info. 7200 IN A 125.46.22.158

 AUTHORITY:
lxdns.info. 86400 IN NS ns3.lxdns.info.
lxdns.info. 86400 IN NS ns1.lxdns.info.
lxdns.info. 86400 IN NS ns4.lxdns.info.
lxdns.info. 86400 IN NS ns2.lxdns.info.
lxdns.info. 86400 IN NS ns5.lxdns.info.

 ADDITIONAL:
ns3.lxdns.info. 7200 IN A 183.131.192.93
ns1.lxdns.info. 7200 IN A 183.131.192.82
ns4.lxdns.info. 7200 IN A 111.178.234.69
ns2.lxdns.info. 7200 IN A 125.46.22.158
ns5.lxdns.info. 7200 IN A 14.119.111.147


In [328]:
item = response8.authority[0]
type(item)

dns.rrset.RRset

In [254]:
item.to_text()

'dns2.lxdns.info. 7200 IN A 125.46.22.158'

In [246]:
response8.additional.append(item)

In [247]:
print_response(response8)


 QUESTION:
dns2.lxdns.info. IN A

 ANSWER:
dns2.lxdns.info. 7200 IN A 125.46.22.158

 AUTHORITY:
lxdns.info. 86400 IN NS ns5.lxdns.info.
lxdns.info. 86400 IN NS ns2.lxdns.info.
lxdns.info. 86400 IN NS ns1.lxdns.info.
lxdns.info. 86400 IN NS ns4.lxdns.info.
lxdns.info. 86400 IN NS ns3.lxdns.info.

 ADDITIONAL:
ns5.lxdns.info. 7200 IN A 14.119.111.147
ns2.lxdns.info. 7200 IN A 125.46.22.158
ns1.lxdns.info. 7200 IN A 183.131.192.82
ns4.lxdns.info. 7200 IN A 111.178.234.69
ns3.lxdns.info. 7200 IN A 183.131.192.93
dns2.lxdns.info. 7200 IN A 125.46.22.158


In [230]:
item.to_text()

'dns2.lxdns.info. 7200 IN A 125.46.22.158'

In [256]:
response5.additional.append(item)

In [257]:
print_response(response5)


 QUESTION:
www.163.com.lxdns.com. IN A

 ANSWER:

 AUTHORITY:
lxdns.com. 172800 IN NS dns2.lxdns.info.
lxdns.com. 172800 IN NS dns4.lxdns.info.
lxdns.com. 172800 IN NS dns1.lxdns.org.
lxdns.com. 172800 IN NS dns3.lxdns.org.
lxdns.com. 172800 IN NS dns5.lxdns.org.

 ADDITIONAL:
dns2.lxdns.info. 7200 IN A 125.46.22.158


In [311]:
rrsets = response5.authority[0].items

In [323]:
get_ns_from_authority(response5)

'dns1.lxdns.org.'

In [316]:
def get_ns_from_authority(response):
    '''Get a name server from AUTHORITY.
    In some cases, there is no ADDITIONAL section, just AUTHORITY section!
    
    Args:
        response (dns.message.Message)
        
    Return:
        string: A name of ns server
    '''
    rrsets = response.authority[0].items
    index = random.randint(0, len(rrsets)-1)
    return rrsets[index].to_text()

In [235]:
response5.additional.append(item)

In [262]:
response5.additional[0].items

[<DNS IN A rdata: 125.46.22.158>]

In [249]:
print_response(response5)


 QUESTION:
www.163.com.lxdns.com. IN A

 ANSWER:

 AUTHORITY:
lxdns.com. 172800 IN NS dns2.lxdns.info.
lxdns.com. 172800 IN NS dns4.lxdns.info.
lxdns.com. 172800 IN NS dns1.lxdns.org.
lxdns.com. 172800 IN NS dns3.lxdns.org.
lxdns.com. 172800 IN NS dns5.lxdns.org.

 ADDITIONAL:
dns2.lxdns.info. 7200 IN A 125.46.22.158


In [119]:
response9 = dns.query.udp(b_query, '125.46.22.158')
print_response(response9)


 QUESTION:
www.163.com.lxdns.com. IN A

 ANSWER:
www.163.com.lxdns.com. 600 IN CNAME opthw.xdwscache.speedcdns.com.

 AUTHORITY:

 ADDITIONAL:


In [120]:
d_query = dns.message.make_query('opthw.xdwscache.speedcdns.com', 'A')
response10 = dns.query.udp(d_query, '198.41.0.4')
print_response(response10)


 QUESTION:
opthw.xdwscache.speedcdns.com. IN A

 ANSWER:

 AUTHORITY:
com. 172800 IN NS a.gtld-servers.net.
com. 172800 IN NS b.gtld-servers.net.
com. 172800 IN NS c.gtld-servers.net.
com. 172800 IN NS d.gtld-servers.net.
com. 172800 IN NS e.gtld-servers.net.
com. 172800 IN NS f.gtld-servers.net.
com. 172800 IN NS g.gtld-servers.net.
com. 172800 IN NS h.gtld-servers.net.
com. 172800 IN NS i.gtld-servers.net.
com. 172800 IN NS j.gtld-servers.net.
com. 172800 IN NS k.gtld-servers.net.
com. 172800 IN NS l.gtld-servers.net.
com. 172800 IN NS m.gtld-servers.net.

 ADDITIONAL:
a.gtld-servers.net. 172800 IN A 192.5.6.30
b.gtld-servers.net. 172800 IN A 192.33.14.30
c.gtld-servers.net. 172800 IN A 192.26.92.30
d.gtld-servers.net. 172800 IN A 192.31.80.30
e.gtld-servers.net. 172800 IN A 192.12.94.30
f.gtld-servers.net. 172800 IN A 192.35.51.30
g.gtld-servers.net. 172800 IN A 192.42.93.30
h.gtld-servers.net. 172800 IN A 192.54.112.30
i.gtld-servers.net. 172800 IN A 192.43.172.30
j.gtld-servers.n

In [121]:
response11 = dns.query.udp(d_query, '192.5.6.30')
print_response(response11)


 QUESTION:
opthw.xdwscache.speedcdns.com. IN A

 ANSWER:

 AUTHORITY:
speedcdns.com. 172800 IN NS dns4.speedcdns.info.
speedcdns.com. 172800 IN NS dns5.speedcdns.info.
speedcdns.com. 172800 IN NS dns3.speedcdns.info.
speedcdns.com. 172800 IN NS dns2.speedcdns.info.
speedcdns.com. 172800 IN NS dns1.speedcdns.info.

 ADDITIONAL:


In [122]:
e_query = dns.message.make_query('dns4.speedcdns.info', 'A')
response12 = dns.query.udp(e_query, '198.41.0.4')
print_response(response12)


 QUESTION:
dns4.speedcdns.info. IN A

 ANSWER:

 AUTHORITY:
info. 172800 IN NS a0.info.afilias-nst.info.
info. 172800 IN NS a2.info.afilias-nst.info.
info. 172800 IN NS b0.info.afilias-nst.org.
info. 172800 IN NS b2.info.afilias-nst.org.
info. 172800 IN NS c0.info.afilias-nst.info.
info. 172800 IN NS d0.info.afilias-nst.org.

 ADDITIONAL:
a0.info.afilias-nst.info. 172800 IN A 199.254.31.1
a2.info.afilias-nst.info. 172800 IN A 199.249.113.1
b0.info.afilias-nst.org. 172800 IN A 199.254.48.1
b2.info.afilias-nst.org. 172800 IN A 199.249.121.1
c0.info.afilias-nst.info. 172800 IN A 199.254.49.1
d0.info.afilias-nst.org. 172800 IN A 199.254.50.1
a0.info.afilias-nst.info. 172800 IN AAAA 2001:500:19::1
a2.info.afilias-nst.info. 172800 IN AAAA 2001:500:41::1
b0.info.afilias-nst.org. 172800 IN AAAA 2001:500:1a::1
b2.info.afilias-nst.org. 172800 IN AAAA 2001:500:49::1
c0.info.afilias-nst.info. 172800 IN AAAA 2001:500:1b::1
d0.info.afilias-nst.org. 172800 IN AAAA 2001:500:1c::1


In [124]:
response13 = dns.query.udp(e_query, '199.254.31.1')
print_response(response13)


 QUESTION:
dns4.speedcdns.info. IN A

 ANSWER:

 AUTHORITY:
speedcdns.info. 86400 IN NS ns3.speedcdns.info.
speedcdns.info. 86400 IN NS ns2.speedcdns.info.
speedcdns.info. 86400 IN NS ns5.speedcdns.info.
speedcdns.info. 86400 IN NS ns4.speedcdns.info.
speedcdns.info. 86400 IN NS ns1.speedcdns.info.

 ADDITIONAL:
ns1.speedcdns.info. 86400 IN A 220.243.237.83
ns2.speedcdns.info. 86400 IN A 203.130.63.64
ns3.speedcdns.info. 86400 IN A 203.130.55.52
ns4.speedcdns.info. 86400 IN A 203.130.58.104
ns5.speedcdns.info. 86400 IN A 220.243.194.17


In [146]:
response13.additional[2].items

[<DNS IN A rdata: 203.130.55.52>]

In [336]:
# 220.243.237.83 time out!
# response14 = dns.query.udp(e_query, '220.243.237.83', 5)
response14 = dns.query.udp(e_query, '203.130.63.64', 5)

print_response(response14)


 QUESTION:
dns4.speedcdns.info. IN A

 ANSWER:
dns4.speedcdns.info. 86400 IN A 157.185.164.91

 AUTHORITY:
speedcdns.info. 86400 IN NS ns1.speedcdns.info.
speedcdns.info. 86400 IN NS ns5.speedcdns.info.
speedcdns.info. 86400 IN NS ns2.speedcdns.info.
speedcdns.info. 86400 IN NS ns4.speedcdns.info.
speedcdns.info. 86400 IN NS ns3.speedcdns.info.

 ADDITIONAL:
ns1.speedcdns.info. 7200 IN A 70.39.191.24
ns5.speedcdns.info. 86400 IN A 203.130.63.64
ns2.speedcdns.info. 7200 IN A 157.185.149.133
ns4.speedcdns.info. 7200 IN A 157.185.166.19
ns3.speedcdns.info. 7200 IN A 220.243.254.26


In [333]:
response15 = dns.query.udp(d_query, '157.185.164.91')
print_response(response15)


 QUESTION:
opthw.xdwscache.speedcdns.com. IN A

 ANSWER:
opthw.xdwscache.speedcdns.com. 60 IN A 203.130.53.126

 AUTHORITY:
speedcdns.com. 86400 IN NS dns4.speedcdns.info.
speedcdns.com. 86400 IN NS dns3.speedcdns.info.
speedcdns.com. 86400 IN NS dns2.speedcdns.info.
speedcdns.com. 86400 IN NS dns5.speedcdns.info.
speedcdns.com. 86400 IN NS dns1.speedcdns.info.

 ADDITIONAL:
dns4.speedcdns.info. 86400 IN A 157.185.164.91
dns3.speedcdns.info. 7200 IN A 203.130.53.12
dns2.speedcdns.info. 7200 IN A 220.243.215.18
dns5.speedcdns.info. 86400 IN A 203.130.63.64
dns1.speedcdns.info. 7200 IN A 157.185.170.124


In [79]:
def check_response(response):
    '''Check whether the response has a valid IP address in its ANSWER section.
    
    Args:
        response (dns.message.Message): the response of a single query
    
    Return:
        True or False
    '''
    try:
        ip = get_ip_from_rrset(response.answer[0])
        ipaddress.ip_address(ip)
        return True
    except:
        return False   

In [85]:
print(response6)

id 30407
opcode QUERY
rcode NOERROR
flags QR AA RD
;QUESTION
turner-tls.map.fastly.net. IN A
;ANSWER
turner-tls.map.fastly.net. 30 IN A 151.101.21.67
;AUTHORITY
fastly.net. 7200 IN NS ns1.fastly.net.
fastly.net. 7200 IN NS ns2.fastly.net.
fastly.net. 7200 IN NS ns3.fastly.net.
fastly.net. 7200 IN NS ns4.fastly.net.
;ADDITIONAL


In [88]:
if check_response(response6):
    print('caitao')

caitao


In [83]:
def get_ip_from_rrset(rrset):
    ''' Get IP address from a RRset (from ADDITIONAL section)
    
    Args:
        rrset (A DNS RRset ): contains an IP address that needs to be extracted
        
    Return:
        ip (str): the IP address in the RRset
    '''
    try:
        item = rrset.items[0]
        return item.to_text()
    except Exception as e:
        print('Oops! Some issue with rrset: ', e)

In [78]:
get_ip_from_rrset(response6.answer[0])

'151.101.21.67'

In [13]:
import ipaddress

In [20]:
def valid_ip(address):
    try: 
        ipaddress.ip_address(address)
        return True
    except:
        return False

print(valid_ip('10.10.20.30'))
print(valid_ip('2001:DB8::1'))
print(valid_ip('gibberish'))

True
True
False


In [339]:
for i in range(0, 10):
    try:
        raise_exception(i)
    except:
        continue
    print('caitao', i)

0
caitao 0
caitao 1
2
caitao 2
3
caitao 3
caitao 4
caitao 5
6
caitao 6
caitao 7
caitao 8
9
caitao 9


In [338]:
def raise_exception(i):
    try:
        a, b = 1, 0
        if random.randint(0,1) == 1:
            c = a/b
    except:
        print(i)

In [340]:
e_query = dns.message.make_query('dns1.speedcdns.info', 'A')
response12 = dns.query.udp(e_query, '198.41.0.4')
print_response(response12)


 QUESTION:
dns1.speedcdns.info. IN A

 ANSWER:

 AUTHORITY:
info. 172800 IN NS a0.info.afilias-nst.info.
info. 172800 IN NS a2.info.afilias-nst.info.
info. 172800 IN NS b0.info.afilias-nst.org.
info. 172800 IN NS b2.info.afilias-nst.org.
info. 172800 IN NS c0.info.afilias-nst.info.
info. 172800 IN NS d0.info.afilias-nst.org.

 ADDITIONAL:
a0.info.afilias-nst.info. 172800 IN A 199.254.31.1
a2.info.afilias-nst.info. 172800 IN A 199.249.113.1
b0.info.afilias-nst.org. 172800 IN A 199.254.48.1
b2.info.afilias-nst.org. 172800 IN A 199.249.121.1
c0.info.afilias-nst.info. 172800 IN A 199.254.49.1
d0.info.afilias-nst.org. 172800 IN A 199.254.50.1
a0.info.afilias-nst.info. 172800 IN AAAA 2001:500:19::1
a2.info.afilias-nst.info. 172800 IN AAAA 2001:500:41::1
b0.info.afilias-nst.org. 172800 IN AAAA 2001:500:1a::1
b2.info.afilias-nst.org. 172800 IN AAAA 2001:500:49::1
c0.info.afilias-nst.info. 172800 IN AAAA 2001:500:1b::1
d0.info.afilias-nst.org. 172800 IN AAAA 2001:500:1c::1


In [341]:
response12 = dns.query.udp(e_query, '199.249.113.1')
print_response(response12)


 QUESTION:
dns1.speedcdns.info. IN A

 ANSWER:

 AUTHORITY:
speedcdns.info. 86400 IN NS ns1.speedcdns.info.
speedcdns.info. 86400 IN NS ns2.speedcdns.info.
speedcdns.info. 86400 IN NS ns3.speedcdns.info.
speedcdns.info. 86400 IN NS ns4.speedcdns.info.
speedcdns.info. 86400 IN NS ns5.speedcdns.info.

 ADDITIONAL:
ns1.speedcdns.info. 86400 IN A 220.243.237.83
ns2.speedcdns.info. 86400 IN A 203.130.63.64
ns3.speedcdns.info. 86400 IN A 203.130.55.52
ns4.speedcdns.info. 86400 IN A 203.130.58.104
ns5.speedcdns.info. 86400 IN A 220.243.194.17


In [343]:
response12 = dns.query.udp(e_query, '220.243.237.83', 1)
print_response(response12)

Timeout: The DNS operation timed out.